In [1]:
import numpy as np
from typing import Tuple, List, Dict, Set
from matplotlib import pyplot as plt
import networkx as nx
import random
from heapq import heappop, heappush
from dataclasses import dataclass, field

In [2]:
def lu(m):
    """ LU разложение по "схеме Халецкого" из статьи """
    shape = m.shape
    l = np.zeros(shape, dtype=complex)
    u = np.zeros(shape, dtype=complex)
    for i in range(shape[0]):
        l[i, 0] = m[i, 0]
        u[i, i] = 1
    for i in range(shape[0]):
        for j in range(shape[0]):
            sum = 0
            if (i >= j):
                for k in range(j):
                    sum += l[i, k] * u[k, j]
                l[i, j] = m[i, j] - sum
            else:
                for k in range(j):
                    sum += l[i, k] * u[k, j]
                u[i, j] = (m[i, j] - sum) / l[i, i]
    return l, u


def printPattern(m: np.ndarray, zeroSymbol="0") -> None:
    """ Распечатать схему расположения ненулевых элементов матрицы """
    for i, row in enumerate(m):
        elts = ["x" if elt != 0 else zeroSymbol for elt in row]
        print(f"{i:03}:  " + "  ".join(elts))


In [3]:
class Node:
    """ Узел схемы """
    def __init__(self, i, x: float = 0, label: str = "") -> None:
        self.number = i
        self.x = x
        self.label = label


def graphToMatrix(g: nx.Graph) -> nx.Graph:
    """ Составить матрицу узловых проводимостей для графа схемы """
    sz = len(g.nodes) - 1
    res = np.zeros((sz, sz))
    for n in filter(lambda n: not np.isnan(n.x), g.nodes):
        for n1, n2, d in g.edges(n, data=True):
            if np.isnan(n1.x) or np.isnan(n2.x):
                continue
            v = d["weight"]
            res[n1.number, n2.number] = -v
            res[n.number, n.number] += v
    return res


def dictToGraph(d: Dict[int, Tuple[float, List[int]]]) -> nx.Graph:
    """ 
        Построить граф по списку смежности, заданному словарем.
        Ключи словаря - номера узлов, значения - кортежи, где первый элемент --
        координата узла, а второй -- список смежных узлов.
    """
    g = nx.Graph()
    nodes = {i: Node(i, data[0], str(i)) for i, data in d.items()}
    for n in nodes.values():
        g.add_node(n)
    for n in g.nodes:
        existingNeighbors = g.neighbors(n)
        _, adjNodeIndices = d[n.number]
        for i in adjNodeIndices:
            adjNode = nodes[i]
            if (adjNode not in existingNeighbors):
                g.add_edge(n, adjNode, weight=random.random() * 10)
    return g


def diff(d: Dict[int, Tuple[float, List[int]]], g: nx.Graph) -> str:
    """
        Для отладки. Проверить граф на соответствие списку смежности.
    """
    dd = {}
    mismatchCnt = 0
    for n in g.nodes:
        a = g.neighbors(n)
        dd[n.number] = [_n.number for _n in a]
    for key, data in d.items():
        dList = sorted(data[1])
        ddList = sorted(dd[key])
        if (len(dList) != len(ddList)):
            print(f"{key}: {dList} <--> {ddList}")
            mismatchCnt += 1
        for i, dElt in enumerate(dList):
            ddElt = ddList[i]
            if ddElt != dElt:
                print(f"key = {key}, {ddElt} != {dElt}")
                mismatchCnt += 1
    return "OK" if mismatchCnt == 0 else "Oops"
    


In [4]:
@dataclass(order=True)
class PrioritizedNode:
    """ Обертка для узла. Нужна, чтобы работала стандартная очередь с приоритетом. """
    priority: int
    node: Node = field(compare=False)


def enumerateNodes(g: nx.Graph):
    """ Обойти граф и пронумеровать узлы """
    q = []  # пустой список, который потом станет очередью с приоритетом
    # множество узлов, уже попавших в обработку
    touchedNodes: Set[Node] = set()

    def push(n: Node):  # helper -- положить узел в очередь
        heappush(q, PrioritizedNode(n.x, n))

    def pop() -> Node:  # helper -- вытолкнуть узел из очереди
        return heappop(q).node

    # общий (базовый) узел
    commonNode = next(filter(lambda n: np.isnan(n.x), g.nodes))
    touchedNodes.add(commonNode)
    commonNodeNeighbors = g.neighbors(commonNode)
    for n in commonNodeNeighbors:
        push(n)
        touchedNodes.add(n)

    n = 0
    while len(q) > 0:  # пока в очереди остаются узлы
        node = pop()
        node.number = n
        n += 1
        for neighbor in g.neighbors(node):
            if neighbor not in touchedNodes:
                touchedNodes.add(neighbor)
                push(neighbor)


def printGraphNodes(g: nx.Graph) -> None:
    """ Для отладки. Вывести метки узлов по возрастанию номера узла. """
    print([node.label for node in sorted(g.nodes, key=lambda n: n.number)])


Тесты

In [40]:
# Список смежности для тестовой схемы
d = {
    -1: (np.nan, [6, 23, 42]),
    0:  (0,  [1, 2, 3, 4]),
    1:  (0,  [0, 2, 3, 4]),
    2:  (20, [0, 1, 3]),
    3:  (20, [0, 1, 2]),
    4:  (20, [0, 1, 5]),
    5:  (20, [4, 6, 7]),
    6:  (20, [-1, 5, 7]),
    7:  (20, [5, 6, 8]),
    8:  (20, [7, 9, 10]),
    9:  (40, [8, 10, 11, 12, 13, 14]),
    10: (40, [8, 9, 11, 12, 13, 14]),
    11: (20, [9, 10, 12]),
    12: (20, [9, 10, 11]),
    13: (85, [9, 10, 14, 15]),
    14: (85, [9, 10, 13, 15]),
    15: (100, [13, 14, 16]),
    16: (100, [15, 17, 18]),
    17: (115, [16, 18, 19, 20]),
    18: (115, [16, 17, 19, 20]),
    19: (200, [17, 18, 20, 21, 26, 27]),
    20: (200, [17, 18, 19, 21, 26, 27]),
    21: (220, [19, 20, 22]),
    22: (220, [21, 23, 24]),
    23: (220, [-1, 22, 24]),
    24: (220, [22, 23, 25]),
    25: (220, [24, 28, 29]),
    26: (220, [19, 20, 27]),
    27: (220, [19, 20, 26]),
    28: (240, [25, 29, 30, 31, 32, 33]),
    29: (240, [25, 28, 30, 31, 32, 33]),
    30: (220, [28, 29, 31]),
    31: (220, [28, 29, 30]),
    32: (285, [28, 29, 33, 34]),
    33: (285, [28, 29, 32, 34]),
    34: (300, [32, 33, 35]),
    35: (300, [34, 36, 37]),
    36: (315, [35, 37, 38, 39]),
    37: (315, [35, 36, 38, 39]),
    38: (400, [36, 37, 39, 40, 45, 46]),
    39: (400, [36, 37, 38, 40, 45, 46]),
    40: (420, [38, 39, 41]),
    41: (420, [40, 42, 43]),
    42: (420, [-1, 41, 43]),
    43: (420, [41, 42, 44]),
    44: (420, [43, 47, 48]),
    45: (420, [38, 39, 46]),
    46: (420, [38, 39, 45]),
    47: (440, [48, 49, 50, 44]),
    48: (440, [47, 49, 50, 44]),
    49: (420, [47, 48, 50]),
    50: (420, [47, 48, 49]),
    51: (220, [18, 19, 4, 3]),
}


In [45]:
g = dictToGraph(d)  # построить граф по списку смежности
print(f"Проверка соответствия графа списку смежности: {diff(d, g)}")
m = graphToMatrix(g)  # построить по графу матрицу узловых проводимостей
print(f"|m| = {np.linalg.det(m)}") # на всякий случай убедиться, что матрица невырождена
print(f"m: {np.count_nonzero(m)}") # кол. ненулевых элементов в матрице m
l, u = lu(m)
print(f"lu: {np.count_nonzero(l + u)}") # кол. ненулевых элементов в LU разложении матрицы m (суммарно)

printPattern(m, zeroSymbol=" ")


3: [0, 1, 2] <--> [0, 1, 2, 51]
4: [0, 1, 5] <--> [0, 1, 5, 51]
18: [16, 17, 19, 20] <--> [16, 17, 19, 20, 51]
19: [17, 18, 20, 21, 26, 27] <--> [17, 18, 20, 21, 26, 27, 51]
Проверка соответствия графа списку смежности: Oops
|m| = -5.651789782458552e+35
m: 246
lu: 410
000:  x  x  x  x  x                                                                                                                                             
001:  x  x  x  x  x                                                                                                                                             
002:  x  x  x  x                                                                                                                                                
003:  x  x  x  x                                                                                                                                               x
004:  x  x        x  x                                                                 

In [49]:
# Перемешать номера узлов и посмотреть разницу
nodes = list(filter(lambda n: not np.isnan(n.x), g.nodes))
random.shuffle(nodes)
nodes.sort(key=lambda n: len(list(g.neighbors(n))))
for i, n in enumerate(nodes):
    n.number = i

m1 = graphToMatrix(g)

print(f"m: {np.count_nonzero(m1)}")
l, u = lu(m1)
print(f"lu: {np.count_nonzero(l + u)}")
printPattern(m1, zeroSymbol=" ")
printPattern(l, zeroSymbol=" ")


m: 246
lu: 380
000:  x                    x                                                                                                                    x              x
001:     x           x                                                                                            x        x                                    
002:        x                                                  x                                                                             x     x            
003:           x  x        x                                                                                                                                    
004:           x  x        x                                            x                                                                                       
005:     x           x                                                                                x        x                                                
006:               

In [50]:
# Теперь снова пронумеровать узлы, на этот раз с помощью функции enumerateNodes
enumerateNodes(g)
m2 = graphToMatrix(g)
l, u = lu(m2)
print(f"m2: {np.count_nonzero(m2)}")
print(f"lu: {np.count_nonzero(l + u)}")
printGraphNodes(g)
printPattern(m2, " ")
printPattern(l, zeroSymbol=" ")

m2: 246
lu: 320
['-1', '6', '5', '7', '4', '0', '1', '2', '3', '8', '9', '11', '12', '10', '13', '14', '15', '16', '17', '18', '20', '19', '26', '27', '51', '23', '21', '22', '24', '25', '28', '30', '31', '29', '33', '32', '34', '35', '36', '37', '38', '39', '40', '46', '45', '42', '41', '43', '44', '47', '49', '50', '48']
000:  x  x  x                                                                                                                                                   
001:  x  x  x  x                                                                                                                                                
002:  x  x  x                 x                                                                                                                                 
003:     x     x  x  x                                                     x                                                                                    
004:           x  x  x  x  x   